#Project 1
## Group 2 Hepatitis
## SECP3133-02 High Performance Data Processing
### Group members:
##### LIM JING YONG A22EC0182
##### LEE SOON DER A22EC0065
##### JASLENE YU A22EC0171
##### NIK ZULAIKHAA A22EC


In [ ]:
# STEP 1: Install Selenium
!pip install -q selenium

# STEP 2: Download a working version of Chromium and ChromeDriver
!wget -q https://storage.googleapis.com/chromium-browser-snapshots/Linux_x64/1140136/chrome-linux.zip
!wget -q https://storage.googleapis.com/chromium-browser-snapshots/Linux_x64/1140136/chromedriver_linux64.zip

# STEP 3: Unzip and move
!unzip -q chrome-linux.zip
!unzip -q chromedriver_linux64.zip
!mv chrome-linux /usr/local/chrome
!mv chromedriver_linux64/chromedriver /usr/local/chromedriver
!chmod +x /usr/local/chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 25.2 MB/s eta 0:00:00


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import time
import random

# Setup Chrome driver
os.environ["PATH"] += ":/usr/local/chrome:/usr/local/chromedriver"
chrome_options = Options()
chrome_options.binary_location = "/usr/local/chrome/chrome"
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

# List of fake user-agent strings
fake_user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 Edge/17.17134",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36 Edge/18.18363"
]

# Randomly pick a fake user-agent
user_agent = random.choice(fake_user_agents)
chrome_options.add_argument(f"user-agent={user_agent}")

driver = webdriver.Chrome(service=Service("/usr/local/chromedriver"), options=chrome_options)

In [ ]:
# Wrapper selectors
card_wrappers = [
    "PremiumCardstyle__CardWrapper-sywKC",
    "FeaturedCardstyle__FeaturedCardWrapper-jxoTMp",
    "BasicCardstyle__BasicCardWrapper-iiXerd"
]

title_selector = [
    "h2[class^='PremiumCardstyle__TitleWrapper']",
    "h2[class^='FeaturedCardstyle__TitleWrapper']",
    "h2[class^='BasicCardstyle__TitleWrapper']"
]

price_selector = [
    "li[class^='ListingPricestyle__ItemWrapper']",
    "li[class^='ListingPricestyle__ItemWrapper']"
]

location_selector = [
    "div[class^='PremiumCardstyle__AddressWrapper']",
    "div[class^='FeaturedCardstyle__AddressWrapper']",
    "div[class^='BasicCardstyle__AddressWrapper']"
]

details_selector = [
    "p[class^='ListingAttributesstyle__ListingAttrsDescriptionItemWrapper']",
    "p[class^='ListingAttributesstyle__ListingAttrsDescriptionItemWrapper']"
]

agent_selector = [
    "div#listing-heading-title[class^='ListingHeadingstyle__HeadingTitle-cGiSJQ']",
    "div.heading-name"
]

In [ ]:
# Wait for main container to load
def wait_for_any_class(driver, classes, timeout=60):
    WebDriverWait(driver, timeout).until(
        lambda d: any(d.find_elements(By.CLASS_NAME, cls) for cls in classes)
    )

In [ ]:
# CSS Selector extracting data
def extract_from_selectors(card, selectors):
    for selector in selectors:
        element = card.select_one(selector)
        if element:
            return element.get_text(strip=True)
    return None

In [ ]:
# Defining scraping function
def scrape_iproperty_area(area_slug, pages_to_scrape=100):
    base_url = f"https://www.iproperty.com.my/sale/{area_slug}/all-residential/"
    driver.get(base_url)
    listings = []

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
    except Exception as e:
        print(f"[{area_slug}] Failed to load page: {e}")
        return listings

    for page in range(1, pages_to_scrape + 1):
        try:
            wait_for_any_class(driver, card_wrappers, timeout=90)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            property_cards = []
            for wrapper in card_wrappers:
              property_cards += soup.find_all('div', class_=wrapper)

            if not property_cards:
                print(f"[{area_slug}] Page {page}: No listings found.")
                break

            print(f"[{area_slug}] Page {page}: Found {len(property_cards)} listings")

            for card in property_cards:
                property_title = extract_from_selectors(card, title_selector)
                property_price = extract_from_selectors(card, price_selector)
                property_location = extract_from_selectors(card, location_selector)
                property_details = extract_from_selectors(card, details_selector)
                property_agent = extract_from_selectors(card, agent_selector)

                listings.append({
                    'Area': area_slug,
                    'Property Title': property_title,
                    'Property Price': property_price,
                    'Property Location': property_location,
                    'Property Details': property_details,
                    'Property Agent': property_agent,
                    'Source URL': base_url
                })

            if page < pages_to_scrape:
              try:
                next_btn = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, "//a[@aria-label='Go to next page']"))
                    )

                # Check if the button is actually clickable (enabled and visible)
                if next_btn.is_enabled() and next_btn.is_displayed():
                  next_btn.click()
                  time.sleep(random.uniform(3, 5))
                else:
                  print(f"[{area_slug}] No next page, ending scrape early.")
                  break

              except Exception:
                  print(f"[{area_slug}] Next page not available or failed to load. Ending scrape early.")
                  break

        except Exception as e:
            print(f"[{area_slug}] Error scraping page {page}: {e}")
            with open(f"error_page_{area_slug}_page_{page}.html", "w", encoding="utf-8") as f:
                f.write(driver.page_source)
            continue

    return listings

In [ ]:
# Method to save data to csv
def save_to_csv(data, filename="iproperty_listings.csv"):
    if not data:
        print("No data to save.")
        return

    keys = data[0].keys()
    with open(filename, "w", newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)
    print(f"\nSaved {len(data)} listings to '{filename}'")

In [ ]:
area_slugs = [
    "perlis-zop7y",
    "kedah-x5i6n",
    "sarawak-dh4eg",
    "sabah-cc02j",
    "pahang-9ycjt",
    "tambun-z20xa",
    "menglembu-ypm1e",
    "bidor-x843z",
    "seri-iskandar-tydb0",
    "ayer-tawar-tgq9o",
    "kuala-kangsar-s96fp",
    "tanjung-malim-r82wm",
    "lahat-pvsye",
    "ipoh-nnv3d",
    "kuala-kurau-iqkyi",
    "chemor-ifo7p",
    "batu-gajah-hhggp",
    "lumut-gdr8e",
    "seri-manjung-feebl",
    "selama-clb3f",
    "teluk-intan-chiea",
    "gopeng-cfiny",
    "sitiawan-b0ok3",
    "pusing-9991j",
    "sungai-siput-8bx0c",
    "bagan-serai-8ahy1",
    "ulu-kinta-7y8np",
    "simpang-pulai-5bgtp",
    "taiping-3z1t1",
    "kamunting-1yj43",
    "parit-buntar-127xj",
    "kampar-0okpe",
    "sungai-gadut-ypcrs",
    "senawang-y1egi",
    "seremban-wnzhv",
    "sikamat-ujmax",
    "mantin-u79z9",
    "seremban-2-so8bw",
    "bukit-kepayang-r48pp",
    "rantau-qvkoo",
    "labu-osnd3",
    "bandar-sri-sendayan-ljmsj",
    "nilai-j9igx",
    "lenggeng-j5fnd",
    "bahau-hbqwf",
    "tampin-g47kj",
    "port-dickson-f3hac",
    "telok-kemang-dshtd",
    "bandar-enstek-c7jvu",
    "seremban-jaya-8voeq",
    "lukut-6m3hh",
    "ampangan-46taw",
    "rasah-19o14",
    "klebang-xyit8",
    "tanjong-minyak-xgv7z",
    "bukit-baru-x0xrv",
    "melaka-city-p6ijx",
    "semabok-oxmhq",
    "krubong-obtkf",
    "durian-tunggal-vjuys",
    "bukit-katil-j6nla",
    "ujong-pasir-e5puc",
    "batu-berendam-c99rr",
    "ayer-keroh-8tikz",
    "alor-gajah-8e9ht",
    "paya-rumput-7q4h4",
    "cheng-0ideo",
    "bertam-1emd5",
    "presint-11-ygfe1",
    "presint-12-rfq2w",
    "presint-8-ms7ry",
    "presint-18-75biu",
    "putrajaya-0otl5",
    "terengganu-0eu1z",
    "tangkak-yl93j",
    "masai-xovy7",
    "skudai-wulur",
    "johor-bahru-v2xvv",
    "sedenak-uzzgf",
    "segamat-sa2kb",
    "tebrau-rpr9y",
    "pengerang-rdkee",
    "pekan-nenas-qy2y1",
    "perling-o65ld",
    "simpang-rengam-neqgt",
    "muar-mn1yx",
    "kluang-kf9qw",
    "batu-pahat-k9p20",
    "kulai-aeugp",
    "pontian-9bc1t",
    "plentong-95n8e",
    "permas-jaya-8sa0f",
    "tampoi-87xsu",
    "senai-80wyc",
    "pasir-gudang-3w63z",
    "iskandar-puteri-nusajaya-3sj6s",
    "ulu-tiram-38ct7",
    "kota-tinggi-2p35o",
    "gelang-patah-0wuj0",
    "kelantan-8j2mm",
    "kuala-selangor-zgfgu",
    "ulu-langat-y8h9k",
    "puncak-alam-y6auy",
    "mutiara-damansara-xd4bk",
    "petaling-jaya-xc3lb",
    "damansara-perdana-wmgco",
    "bukit-raja-vqvar",
    "tanjong-duabelas-vpngu",
    "banting-sgpso",
    "kuang-qsed8",
    "cheras-qino1",
    "ulu-yam-puu8m",
    "shah-alam-oc15c",
    "subang-jaya-o9c69",
    "klang-nzanj",
    "jenjarom-nv2nl",
    "bandar-sri-damansara-muy33",
    "bandar-kinrara-mlvqy",
    "bandar-sungai-long-mexm6",
    "rawang-lxaou",
    "semenyih-lk99d",
    "selayang-lczck",
    "seri-kembangan-kz4oo",
    "bangi-kwtcc",
    "batu-caves-j16mu",
    "ulu-kelang-i747e",
    "setia-alam-hvrge",
    "saujana-hb944",
    "serendah-gegei",
    "telok-panglima-garang-g8npr",
    "subang-dy92a",
    "saujana-utama-dy4pr",
    "kajang-crp6q",
    "sungai-buloh-c4pye",
    "puchong-ai5y2",
    "beranang-afaef",
    "dengkil-a4la0",
    "ampang-9z0mc",
    "sepang-9vrnn",
    "port-klang-pelabuhan-klang-9mai4",
    "balakong-9e0sz",
    "batu-arang-8t6m4",
    "glenmarie-8mgv5",
    "kepong-7xofr",
    "cyberjaya-7pmtb",
    "hulu-langat-7kfi3",
    "kapar-7hla9",
    "kota-damansara-6jbfp",
    "ara-damansara-662zm",
    "kuala-kubu-bharu-489ni",
    "gombak-3urzp",
    "bandar-utama-3injd",
    "kayu-ara-3dlkp",
    "batang-kali-30ugd",
    "damansara-damai-2bfri",
    "tropicana-22kyq",
    "ijok-0jp17",
    "sunway-05blf",
    "sungai-besi-zeaie",
    "seputeh-u27eo",
    "setapak-op878",
    "bukit-tunku-kenny-hills-mbqas",
    "kl-eco-city-jzhyc",
    "damansara-heights-f3k46",
    "ulu-kelang-ddn97",
    "gombak-bld8d",
    "sri-petaling-aopbi",
    "sungai-penchala-4x7k7",
    "dutamas-2k4qk",
    "jalan-klang-lama-old-klang-road-15jqp",
    "kl-sentral-zbs4q",
    "segambut-tr890",
    "jinjang-oghco",
    "titiwangsa-lk4fv",
    "kepong-jp1ux",
    "sentul-evxyj",
    "sri-hartamas-dd0eh",
    "salak-selatan-b8yzo",
    "keramat-9vnxt",
    "bandar-menjalara-4e6kw",
    "taman-tun-dr-ismail-295cx",
    "taman-desa-1244h",
    "ampang-zbpv1",
    "jalan-kuching-t49fq",
    "sunway-spk-nirhe",
    "cheras-ku7cq",
    "bukit-jalil-ijzo5",
    "desa-parkcity-eqs5n",
    "bukit-kiara-d6i3r",
    "bangsar-av65k",
    "wangsa-maju-8810u",
    "salak-south-45vsg",
    "jalan-ipoh-28ruu",
    "batu-caves-11rvn",
    "bandar-tasik-selatan-v788g",
    "mont-kiara-q9qzw",
    "setiawangsa-mhnc3",
    "kl-city-centre-kpycl",
    "brickfields-fh927",
    "pantai-dx9y0",
    "desa-petaling-cept2",
    "kuchai-lama-appqg",
    "country-heights-damansara-7fszh",
    "kampung-kerinchi-bangsar-south-3hrks",
    "mid-valley-city-20ada",
    "butterworth-zq28a",
    "seberang-jaya-vntw0",
    "bayan-lepas-rkb3o",
    "kepala-batas-kgj0s",
    "sungai-ara-h8sy4",
    "gertak-sanggul-cmqq3",
    "bukit-minyak-av7rk",
    "batu-kawan-9222j",
    "telok-kumbar-5zgzb",
    "nibong-tebal-3tl4k",
    "teluk-bahang-xohvt",
    "ayer-itam-pyg2j",
    "george-town-ju7lu",
    "tanjung-bungah-gxdau",
    "penaga-bvtyc",
    "gurney-drive-9zadk",
    "kubang-semang-8wbel",
    "gelugor-5z75o",
    "tasek-gelugor-24t36",
    "teluk-kumbar-wvpt7",
    "batu-maung-s50h9",
    "balik-pulau-pri9r",
    "sungai-dua-ipe9t",
    "jelutong-dwbmk",
    "bukit-jambul-bhv9m",
    "perai-9ouyc",
    "pulau-tikus-8bfe6",
    "bukit-mertajam-4fw2q",
    "juru-wudu8",
    "seberang-perai-s1xcp",
    "bayan-baru-l7c3z",
    "tanjung-tokong-io38f",
    "permatang-pauh-d14x7",
    "sungai-bakap-aw5no",
    "sungai-jawi-933fv",
    "batu-feringghi-6r1me",
    "simpang-ampat-41pqd"
]

In [ ]:
# Main Method
if __name__ == "__main__":

    all_listings = []

    for slug in area_slugs:
        driver.delete_all_cookies()
        area_data = scrape_iproperty_area(slug, pages_to_scrape=100)
        all_listings.extend(area_data)

        save_to_csv(all_listings, filename="iproperty_all_areas.csv")

        time.sleep(random.uniform(5, 10))

    save_to_csv(all_listings, filename="iproperty_all_areas.csv")
    driver.quit()

[perlis-zop7y] Page 1: Found 20 listings
[perlis-zop7y] Page 2: Found 8 listings
[perlis-zop7y] Next page not available or failed to load. Ending scrape early.

Saved 28 listings to 'iproperty_all_areas.csv'
[kedah-x5i6n] Page 1: Found 20 listings
[kedah-x5i6n] Page 2: Found 20 listings
[kedah-x5i6n] Page 3: Found 20 listings
[kedah-x5i6n] Page 4: Found 20 listings
[kedah-x5i6n] Page 5: Found 20 listings
[kedah-x5i6n] Page 6: Found 20 listings
[kedah-x5i6n] Page 7: Found 20 listings
[kedah-x5i6n] Page 8: Found 20 listings
[kedah-x5i6n] Page 9: Found 20 listings
[kedah-x5i6n] Page 10: Found 20 listings
[kedah-x5i6n] Page 11: Found 20 listings
[kedah-x5i6n] Page 12: Found 20 listings
[kedah-x5i6n] Page 13: Found 20 listings
[kedah-x5i6n] Page 14: Found 20 listings
[kedah-x5i6n] Page 15: Found 20 listings
[kedah-x5i6n] Page 16: Found 20 listings
[kedah-x5i6n] Page 17: Found 20 listings
[kedah-x5i6n] Page 18: Found 20 listings
[kedah-x5i6n] Page 19: Found 20 listings
[kedah-x5i6n] Page 20:

KeyboardInterrupt: 